Going to download data from https://www.unb.ca/cic/datasets/ids-2018.html

But let's see first what's in the bucket:

In [1]:
!aws s3 ls --no-sign-request "s3://cse-cic-ids2018" --recursive --human-readable --summarize

2018-10-10 13:52:09    0 Bytes Original Network Traffic and Log data/
2018-10-10 13:52:23    0 Bytes Original Network Traffic and Log data/Friday-02-03-2018/
2018-10-10 14:00:39  225.8 MiB Original Network Traffic and Log data/Friday-02-03-2018/logs.zip
2018-10-10 14:00:51   41.7 GiB Original Network Traffic and Log data/Friday-02-03-2018/pcap.zip
2018-10-10 13:52:34    0 Bytes Original Network Traffic and Log data/Friday-16-02-2018/
2018-10-10 14:45:49  148.1 MiB Original Network Traffic and Log data/Friday-16-02-2018/logs.zip
2018-10-10 14:46:01   35.9 GiB Original Network Traffic and Log data/Friday-16-02-2018/pcap.zip
2018-10-10 13:52:41    0 Bytes Original Network Traffic and Log data/Friday-23-02-2018/
2018-10-10 14:46:10  199.8 MiB Original Network Traffic and Log data/Friday-23-02-2018/logs.zip
2018-10-10 14:46:31   55.0 GiB Original Network Traffic and Log data/Friday-23-02-2018/pcap.zip
2018-10-10 13:52:47    0 Bytes Original Network Traffic and Log data/Thursday-01-03-2018/


And now download only the csv files:

In [2]:
!aws s3 cp --no-sign-request "s3://cse-cic-ids2018/Processed Traffic Data for ML Algorithms/" data/ --recursive

download: s3://cse-cic-ids2018/Processed Traffic Data for ML Algorithms/Thursday-01-03-2018_TrafficForML_CICFlowMeter.csv to data/Thursday-01-03-2018_TrafficForML_CICFlowMeter.csv
download: s3://cse-cic-ids2018/Processed Traffic Data for ML Algorithms/Friday-16-02-2018_TrafficForML_CICFlowMeter.csv to data/Friday-16-02-2018_TrafficForML_CICFlowMeter.csv
download: s3://cse-cic-ids2018/Processed Traffic Data for ML Algorithms/Friday-02-03-2018_TrafficForML_CICFlowMeter.csv to data/Friday-02-03-2018_TrafficForML_CICFlowMeter.csv
download: s3://cse-cic-ids2018/Processed Traffic Data for ML Algorithms/Friday-23-02-2018_TrafficForML_CICFlowMeter.csv to data/Friday-23-02-2018_TrafficForML_CICFlowMeter.csv
download: s3://cse-cic-ids2018/Processed Traffic Data for ML Algorithms/Thursday-15-02-2018_TrafficForML_CICFlowMeter.csv to data/Thursday-15-02-2018_TrafficForML_CICFlowMeter.csv
download: s3://cse-cic-ids2018/Processed Traffic Data for ML Algorithms/Wednesday-28-02-2018_TrafficForML_CICFlo

In [6]:
!ls -l --block-size=M data

total 6568M
-rw-rw-r-- 1 driss driss  337M oct.  11  2018 Friday-02-03-2018_TrafficForML_CICFlowMeter.csv
-rw-rw-r-- 1 driss driss  319M oct.  11  2018 Friday-16-02-2018_TrafficForML_CICFlowMeter.csv
-rw-rw-r-- 1 driss driss  366M oct.  11  2018 Friday-23-02-2018_TrafficForML_CICFlowMeter.csv
-rw-rw-r-- 1 driss driss 3868M oct.  11  2018 Thuesday-20-02-2018_TrafficForML_CICFlowMeter.csv
-rw-rw-r-- 1 driss driss  103M oct.  11  2018 Thursday-01-03-2018_TrafficForML_CICFlowMeter.csv
-rw-rw-r-- 1 driss driss  359M oct.  11  2018 Thursday-15-02-2018_TrafficForML_CICFlowMeter.csv
-rw-rw-r-- 1 driss driss  365M oct.  11  2018 Thursday-22-02-2018_TrafficForML_CICFlowMeter.csv
-rw-rw-r-- 1 driss driss  342M oct.  11  2018 Wednesday-14-02-2018_TrafficForML_CICFlowMeter.csv
-rw-rw-r-- 1 driss driss  314M oct.  11  2018 Wednesday-21-02-2018_TrafficForML_CICFlowMeter.csv
-rw-rw-r-- 1 driss driss  200M oct.  11  2018 Wednesday-28-02-2018_TrafficForML_CICFlowMeter.csv


In [ ]:
# dates_by_datasets = {
#     "Friday-02-03-2018_TrafficForML_CICFlowMeter.csv": "02-03-2018",
#     "Friday-16-02-2018_TrafficForML_CICFlowMeter.csv": "16-02-2018",
#     "Friday-23-02-2018_TrafficForML_CICFlowMeter.csv": "23-02-2018",
#     "Thuesday-20-02-2018_TrafficForML_CICFlowMeter.csv": "23-02-2018",
#     "Thursday-01-03-2018_TrafficForML_CICFlowMeter.csv": "01-03-2018",
#     "Thursday-15-02-2018_TrafficForML_CICFlowMeter.csv": "15-02-2018",
#     "Thursday-22-02-2018_TrafficForML_CICFlowMeter.csv": "22-02-2018",
#     "Wednesday-14-02-2018_TrafficForML_CICFlowMeter.csv": "14-02-2018",
#     "Wednesday-21-02-2018_TrafficForML_CICFlowMeter.csv": "21-02-2018",
#     "Wednesday-28-02-2018_TrafficForML_CICFlowMeter.csv": "28-02-2018",
# }

In [1]:
import pandas as pd

It seems that dataset "Thuesday-20-02-2018_TrafficForML_CICFlowMeter.csv" is too heavy

Let's split it first.

In [8]:
huge_df = pd.read_csv("data/Thuesday-20-02-2018_TrafficForML_CICFlowMeter.csv")

In [9]:
nb_of_chunks = 10
start, chunk_size = 0, int(huge_df.shape[0] / nb_of_chunks)

for chunk in range(nb_of_chunks):
    huge_df.iloc[start: start + (chunk_size)].to_csv(
        'data/Thuesday-20-02-2018_TrafficForML_CICFlowMeter_{}.csv'.format(chunk),
        index=False
    )
    start += chunk_size

In [10]:
!ls -l --block-size=M data

total 10620M
-rw-rw-r-- 1 driss driss  337M oct.  11  2018 Friday-02-03-2018_TrafficForML_CICFlowMeter.csv
-rw-rw-r-- 1 driss driss  319M oct.  11  2018 Friday-16-02-2018_TrafficForML_CICFlowMeter.csv
-rw-rw-r-- 1 driss driss  366M oct.  11  2018 Friday-23-02-2018_TrafficForML_CICFlowMeter.csv
-rw-rw-r-- 1 driss driss  384M janv. 16 21:59 Thuesday-20-02-2018_TrafficForML_CICFlowMeter_0.csv
-rw-rw-r-- 1 driss driss  407M janv. 16 21:59 Thuesday-20-02-2018_TrafficForML_CICFlowMeter_1.csv
-rw-rw-r-- 1 driss driss  405M janv. 16 22:00 Thuesday-20-02-2018_TrafficForML_CICFlowMeter_2.csv
-rw-rw-r-- 1 driss driss  406M janv. 16 22:00 Thuesday-20-02-2018_TrafficForML_CICFlowMeter_3.csv
-rw-rw-r-- 1 driss driss  407M janv. 16 22:00 Thuesday-20-02-2018_TrafficForML_CICFlowMeter_4.csv
-rw-rw-r-- 1 driss driss  407M janv. 16 22:01 Thuesday-20-02-2018_TrafficForML_CICFlowMeter_5.csv
-rw-rw-r-- 1 driss driss  406M janv. 16 22:01 Thuesday-20-02-2018_TrafficForML_CICFlowMeter_6.csv
-rw-rw-r-- 1 driss 

In [11]:
!rm data/Thuesday-20-02-2018_TrafficForML_CICFlowMeter.csv

In [12]:
csv_files = [
    "Friday-02-03-2018_TrafficForML_CICFlowMeter.csv",
    "Friday-16-02-2018_TrafficForML_CICFlowMeter.csv",
    "Friday-23-02-2018_TrafficForML_CICFlowMeter.csv",
    "Thuesday-20-02-2018_TrafficForML_CICFlowMeter_0.csv",
    "Thursday-01-03-2018_TrafficForML_CICFlowMeter.csv",
    "Thursday-15-02-2018_TrafficForML_CICFlowMeter.csv",
    "Thursday-22-02-2018_TrafficForML_CICFlowMeter.csv",
    "Wednesday-14-02-2018_TrafficForML_CICFlowMeter.csv",
    "Wednesday-21-02-2018_TrafficForML_CICFlowMeter.csv",
    "Wednesday-28-02-2018_TrafficForML_CICFlowMeter.csv",
    "Thuesday-20-02-2018_TrafficForML_CICFlowMeter_1.csv",
    "Thuesday-20-02-2018_TrafficForML_CICFlowMeter_2.csv",
    "Thuesday-20-02-2018_TrafficForML_CICFlowMeter_3.csv",
    "Thuesday-20-02-2018_TrafficForML_CICFlowMeter_4.csv",
    "Thuesday-20-02-2018_TrafficForML_CICFlowMeter_5.csv",
    "Thuesday-20-02-2018_TrafficForML_CICFlowMeter_6.csv",
    "Thuesday-20-02-2018_TrafficForML_CICFlowMeter_7.csv",
    "Thuesday-20-02-2018_TrafficForML_CICFlowMeter_8.csv",
    "Thuesday-20-02-2018_TrafficForML_CICFlowMeter_9.csv",
]

Create a dataframe object

Let's now explore labels distribution and memory usage for each datasets

In [13]:
labels_dist_by_dataset, tot_mem = {}, 0

for csv in csv_files:

    print('Loading', csv)
    tmp_df = pd.read_csv("data/{}".format(csv))
    tmp_dist = (tmp_df['Label'].value_counts(normalize=True) * 100).to_dict()
    tmp_mem = tmp_df.memory_usage(index=True).sum()
    tot_mem += tmp_mem
    tmp_dist['memory_usage'] = tmp_mem
    print(tmp_dist)
    labels_dist_by_dataset[csv] = tmp_dist
    

Loading Friday-02-03-2018_TrafficForML_CICFlowMeter.csv
{'Benign': 72.70667334239324, 'Bot': 27.293326657606755, 'memory_usage': 671088128}
Loading Friday-16-02-2018_TrafficForML_CICFlowMeter.csv


/home/driss/miniconda3/envs/ids-ml/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3146: DtypeWarning: Columns (0,1,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


{'DoS attacks-Hulk': 44.051403094676104, 'Benign': 42.607538802660756, 'DoS attacks-SlowHTTPTest': 13.340962735140549, 'Label': 9.536752259018191e-05, 'memory_usage': 671088128}
Loading Friday-23-02-2018_TrafficForML_CICFlowMeter.csv
{'Benign': 99.94602198221396, 'Brute Force -Web': 0.03452304317764585, 'Brute Force -XSS': 0.014400495911117469, 'SQL Injection': 0.005054478697279641, 'memory_usage': 671088128}
Loading Thuesday-20-02-2018_TrafficForML_CICFlowMeter_0.csv
{'DDoS attacks-LOIC-HTTP': 72.48834406459389, 'Benign': 27.511655935406115, 'memory_usage': 534155456}
Loading Thursday-01-03-2018_TrafficForML_CICFlowMeter.csv
{'Benign': 71.88735371838429, 'Infilteration': 28.10509626274066, 'Label': 0.007550018875047188, 'memory_usage': 211920128}
Loading Thursday-15-02-2018_TrafficForML_CICFlowMeter.csv
{'Benign': 94.99339579906064, 'DoS attacks-GoldenEye': 3.9585151276732713, 'DoS attacks-Slowloris': 1.0480890732660992, 'memory_usage': 671088128}
Loading Thursday-22-02-2018_TrafficFo

Normalize memory usage to be more understandable

In [14]:
for d in labels_dist_by_dataset:
    labels_dist_by_dataset[d]['memory_usage'] = round((labels_dist_by_dataset[d]['memory_usage'] / tot_mem) * 100, 3)

Here we go! Now having more representative stats on each dataset for splitting into train, test and validate sets

In [15]:
labels_dist_by_dataset

{'Friday-02-03-2018_TrafficForML_CICFlowMeter.csv': {'Benign': 72.70667334239324,
  'Bot': 27.293326657606755,
  'memory_usage': 6.305},
 'Friday-16-02-2018_TrafficForML_CICFlowMeter.csv': {'DoS attacks-Hulk': 44.051403094676104,
  'Benign': 42.607538802660756,
  'DoS attacks-SlowHTTPTest': 13.340962735140549,
  'Label': 9.536752259018191e-05,
  'memory_usage': 6.305},
 'Friday-23-02-2018_TrafficForML_CICFlowMeter.csv': {'Benign': 99.94602198221396,
  'Brute Force -Web': 0.03452304317764585,
  'Brute Force -XSS': 0.014400495911117469,
  'SQL Injection': 0.005054478697279641,
  'memory_usage': 6.305},
 'Thuesday-20-02-2018_TrafficForML_CICFlowMeter_0.csv': {'DDoS attacks-LOIC-HTTP': 72.48834406459389,
  'Benign': 27.511655935406115,
  'memory_usage': 5.019},
 'Thursday-01-03-2018_TrafficForML_CICFlowMeter.csv': {'Benign': 71.88735371838429,
  'Infilteration': 28.10509626274066,
  'Label': 0.007550018875047188,
  'memory_usage': 1.991},
 'Thursday-15-02-2018_TrafficForML_CICFlowMeter.csv

In [16]:
huge_df['Label'].value_counts(normalize=True) * 100

Benign                    92.751173
DDoS attacks-LOIC-HTTP     7.248827
Name: Label, dtype: float64

Print columns the names of the first columns because I noticed some differences

In [17]:
for csv in csv_files:

    print('Loading', csv)
    tmp_df = pd.read_csv("data/{}".format(csv))
    print(tmp_df.columns[:5])
    

Loading Friday-02-03-2018_TrafficForML_CICFlowMeter.csv
Index(['Dst Port', 'Protocol', 'Timestamp', 'Flow Duration', 'Tot Fwd Pkts'], dtype='object')
Loading Friday-16-02-2018_TrafficForML_CICFlowMeter.csv
Index(['Dst Port', 'Protocol', 'Timestamp', 'Flow Duration', 'Tot Fwd Pkts'], dtype='object')
Loading Friday-23-02-2018_TrafficForML_CICFlowMeter.csv
Index(['Dst Port', 'Protocol', 'Timestamp', 'Flow Duration', 'Tot Fwd Pkts'], dtype='object')
Loading Thuesday-20-02-2018_TrafficForML_CICFlowMeter_0.csv
Index(['Flow ID', 'Src IP', 'Src Port', 'Dst IP', 'Dst Port'], dtype='object')
Loading Thursday-01-03-2018_TrafficForML_CICFlowMeter.csv
Index(['Dst Port', 'Protocol', 'Timestamp', 'Flow Duration', 'Tot Fwd Pkts'], dtype='object')
Loading Thursday-15-02-2018_TrafficForML_CICFlowMeter.csv
Index(['Dst Port', 'Protocol', 'Timestamp', 'Flow Duration', 'Tot Fwd Pkts'], dtype='object')
Loading Thursday-22-02-2018_TrafficForML_CICFlowMeter.csv
Index(['Dst Port', 'Protocol', 'Timestamp', 'Flow

Obviously the datasets are unbalanced. We must therefore find a compromise to have enough "M-Profile" labeled data (malicious) of each datasets in the training set while avoiding any process of false data generation such as oversampling or something else

In [2]:
def format_columns(df):
    df.columns = [col.upper().replace(' ', '_') for col in df.columns]
    df = df.drop(['FLOW_ID', 'SRC_IP', 'DST_IP', 'SRC_PORT'], axis=1, errors='ignore')
    # ...
    return df

In [3]:
from sklearn.model_selection import train_test_split

def train_test_validation_split(df, test_size, val_size, label_column='LABEL'):
    '''
        :return: train_dataset (pandas.DataFrame), 
                 test_dataset (pandas.DataFrame), 
                 validation_dataset (pandas.DataFrame)
    '''
    # Format columns
    df = format_columns(df)
    
    # Splitting X and y from df
    X, y = df[df.columns.difference(['LABEL'])], df['LABEL']
    # Cutting out test set from df
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=test_size, random_state=1)
    # Cutting out val set from train_df
    X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=val_size, random_state=1)
    
    return pd.concat([X_train, y_train], axis=1), pd.concat([X_test, y_test], axis=1), pd.concat([X_val, y_val], axis=1)

So let's keep this files list for creating our datasets:

In [4]:
csv_files = [
    "Friday-02-03-2018_TrafficForML_CICFlowMeter.csv",
    "Friday-16-02-2018_TrafficForML_CICFlowMeter.csv",
    "Friday-23-02-2018_TrafficForML_CICFlowMeter.csv",
    "Thuesday-20-02-2018_TrafficForML_CICFlowMeter_0.csv",
    "Thursday-01-03-2018_TrafficForML_CICFlowMeter.csv",
    "Thursday-15-02-2018_TrafficForML_CICFlowMeter.csv",
    "Thursday-22-02-2018_TrafficForML_CICFlowMeter.csv",
    "Wednesday-14-02-2018_TrafficForML_CICFlowMeter.csv",
    "Wednesday-21-02-2018_TrafficForML_CICFlowMeter.csv",
    "Wednesday-28-02-2018_TrafficForML_CICFlowMeter.csv",
    "Thuesday-20-02-2018_TrafficForML_CICFlowMeter_1.csv",
    "Thuesday-20-02-2018_TrafficForML_CICFlowMeter_2.csv",
    "Thuesday-20-02-2018_TrafficForML_CICFlowMeter_3.csv",
]

In [5]:
train, test, val = pd.DataFrame({}), pd.DataFrame({}), pd.DataFrame({})

for csv in csv_files:

    print('*** Loading', csv)
    
    tmp_train, tmp_test, tmp_val = train_test_validation_split(
        df=pd.read_csv("data/{}".format(csv)),
        test_size=0.3,
        val_size=0.14, # (1 - 0.3) * 0.14 = 0.1
    )
    
    
    train = pd.concat([train, tmp_train])
    print('Train set concatenated: new train dataframe shape: {}'.format(train.shape[0]))
    test = pd.concat([test, tmp_test])
    print('Test set concatenated: new test dataframe shape: {}'.format(test.shape[0]))
    val = pd.concat([val, tmp_val])
    print('Val set concatenated: new test dataframe shape: {}'.format(val.shape[0]))
    

*** Loading Friday-02-03-2018_TrafficForML_CICFlowMeter.csv
Train set concatenated: new train dataframe shape: 631241
Test set concatenated: new test dataframe shape: 314573
Val set concatenated: new test dataframe shape: 102761
*** Loading Friday-16-02-2018_TrafficForML_CICFlowMeter.csv


/home/driss/miniconda3/envs/ids-ml/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3146: DtypeWarning: Columns (0,1,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


Train set concatenated: new train dataframe shape: 1262482
Test set concatenated: new test dataframe shape: 629146
Val set concatenated: new test dataframe shape: 205522
*** Loading Friday-23-02-2018_TrafficForML_CICFlowMeter.csv
Train set concatenated: new train dataframe shape: 1893723
Test set concatenated: new test dataframe shape: 943719
Val set concatenated: new test dataframe shape: 308283
*** Loading Thuesday-20-02-2018_TrafficForML_CICFlowMeter_0.csv
Train set concatenated: new train dataframe shape: 2372236
Test set concatenated: new test dataframe shape: 1182182
Val set concatenated: new test dataframe shape: 386181
*** Loading Thursday-01-03-2018_TrafficForML_CICFlowMeter.csv
Train set concatenated: new train dataframe shape: 2571572
Test set concatenated: new test dataframe shape: 1281520
Val set concatenated: new test dataframe shape: 418632
*** Loading Thursday-15-02-2018_TrafficForML_CICFlowMeter.csv
Train set concatenated: new train dataframe shape: 3202813
Test set co

In [6]:
print(train.shape)
train.head()

(6901162, 80)


,ACK_FLAG_CNT,ACTIVE_MAX,ACTIVE_MEAN,ACTIVE_MIN,ACTIVE_STD,BWD_BLK_RATE_AVG,BWD_BYTS/B_AVG,BWD_HEADER_LEN,BWD_IAT_MAX,BWD_IAT_MEAN,...,SUBFLOW_FWD_BYTS,SUBFLOW_FWD_PKTS,SYN_FLAG_CNT,TIMESTAMP,TOTLEN_BWD_PKTS,TOTLEN_FWD_PKTS,TOT_BWD_PKTS,TOT_FWD_PKTS,URG_FLAG_CNT,LABEL
48911,0,0.0,0.0,0.0,0.0,0,0,92,9156.0,3164.333333,...,326,3,0,02/03/2018 11:26:37,129.0,326,4,3,0,Bot
968389,0,0.0,0.0,0.0,0.0,0,0,152,953134.0,367342.333300,...,1138,8,0,02/03/2018 12:52:16,1581.0,1138,7,8,0,Benign
148507,0,0.0,0.0,0.0,0.0,0,0,92,10104.0,3510.000000,...,326,3,0,02/03/2018 02:27:18,129.0,326,4,3,0,Bot
709046,0,0.0,0.0,0.0,0.0,0,0,172,3424785.0,1087399.857000,...,1435,14,0,02/03/2018 04:22:32,1731.0,1435,8,14,0,Benign
840709,0,0.0,0.0,0.0,0.0,0,0,152,953332.0,262287.333300,...,1128,8,0,02/03/2018 03:56:14,1581.0,1128,7,8,0,Benign


In [7]:
print(test.shape)
test.head()

(3439133, 80)


,ACK_FLAG_CNT,ACTIVE_MAX,ACTIVE_MEAN,ACTIVE_MIN,ACTIVE_STD,BWD_BLK_RATE_AVG,BWD_BYTS/B_AVG,BWD_HEADER_LEN,BWD_IAT_MAX,BWD_IAT_MEAN,...,SUBFLOW_FWD_BYTS,SUBFLOW_FWD_PKTS,SYN_FLAG_CNT,TIMESTAMP,TOTLEN_BWD_PKTS,TOTLEN_FWD_PKTS,TOT_BWD_PKTS,TOT_FWD_PKTS,URG_FLAG_CNT,LABEL
1045953,0,125449.0,125449.0,125449.0,0.0,0,0,232,10000000.0,1009987.3,...,818,8,0,02/03/2018 12:52:25,6559.0,818,11,8,0,Benign
358409,1,0.0,0.0,0.0,0.0,0,0,0,0.0,0.0,...,31,2,1,02/03/2018 09:27:35,0.0,31,0,2,0,Benign
875012,0,0.0,0.0,0.0,0.0,0,0,152,953181.0,267153.0,...,1148,9,0,02/03/2018 03:55:10,1581.0,1148,7,9,0,Benign
555046,0,0.0,0.0,0.0,0.0,0,0,8,0.0,0.0,...,41,1,0,02/03/2018 11:01:44,105.0,41,1,1,0,Benign
272972,1,0.0,0.0,0.0,0.0,0,0,0,0.0,0.0,...,0,2,0,02/03/2018 03:48:35,0.0,0,0,2,0,Bot


In [8]:
print(val.shape)
val.head()

(1123455, 80)


,ACK_FLAG_CNT,ACTIVE_MAX,ACTIVE_MEAN,ACTIVE_MIN,ACTIVE_STD,BWD_BLK_RATE_AVG,BWD_BYTS/B_AVG,BWD_HEADER_LEN,BWD_IAT_MAX,BWD_IAT_MEAN,...,SUBFLOW_FWD_BYTS,SUBFLOW_FWD_PKTS,SYN_FLAG_CNT,TIMESTAMP,TOTLEN_BWD_PKTS,TOTLEN_FWD_PKTS,TOT_BWD_PKTS,TOT_FWD_PKTS,URG_FLAG_CNT,LABEL
923977,0,0.0,0.0,0.0,0.0,0,0,0,0.0,0.000000,...,0,2,0,02/03/2018 01:52:10,0.0,0,0,2,0,Benign
984998,0,0.0,0.0,0.0,0.0,0,0,172,969738.0,582076.857100,...,1438,14,0,02/03/2018 02:17:56,1731.0,1438,8,14,0,Benign
952845,0,0.0,0.0,0.0,0.0,0,0,92,393.0,245.333333,...,148,3,0,02/03/2018 11:50:13,243.0,148,4,3,0,Benign
816873,0,0.0,0.0,0.0,0.0,0,0,8,0.0,0.000000,...,47,1,0,02/03/2018 09:21:43,233.0,47,1,1,0,Benign
910181,0,0.0,0.0,0.0,0.0,0,0,8,0.0,0.000000,...,315,1,0,02/03/2018 11:16:33,302.0,315,1,1,0,Benign


In [11]:
list(train.columns) == list(test.columns) == list(val.columns)

True

And then, export them in csv files for an upcoming exploration, modelization, test and validation....

In [12]:
train.to_csv('train.csv', index=False)

In [13]:
test.to_csv('test.csv', index=False)

In [14]:
val.to_csv('val.csv', index=False)

In [15]:
!ls

'1-collect_&_extract_data.ipynb'   data       train.csv
 2-explore_data.ipynb		   test.csv   val.csv
